# 모델링

## train_test_split

In [1]:
import pandas as pd
#df_fin = pd.read_csv('전처리_final.csv')

In [348]:
df_fin = pd.read_csv('전처리_screen_act있는.csv')

In [349]:
df_fin

,screenCnt,act_n,dist_rank,genre_rank,direct_rank,네이버_기대지수,star_buzz,movie_buzz_naver,google_trend,Target
0,1183.0,121,4,5,67,1274.0,1629,22,773.0,0
1,576.0,50,8,13,67,3838.0,581,19,387.0,0
2,696.0,96,2,13,41,494.0,943,30,616.0,0
3,54.0,12,8,13,94,162.0,111,6,0.0,0
4,106.0,30,3,13,67,800.0,335,5,1674.0,0
...,...,...,...,...,...,...,...,...,...,...
570,1009.0,73,3,13,5,13693.0,1987,219,947.0,1
571,621.0,70,2,13,72,10731.0,1196,124,886.0,0
572,838.0,148,4,1,5,4941.0,1413,379,0.0,1
573,14.0,32,7,8,100,247.0,147,25,459.0,0


In [350]:
df_fin = df_fin.drop(['screenCnt', 'act_n'], axis=1)

In [351]:
y = df_fin['Target']
x = df_fin.drop('Target', axis=1)

In [352]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=100)

In [116]:
# train_test 정규화, 정규화하는 것이 더 좋은 결과를 낳음
from sklearn.preprocessing import StandardScaler
 
sc = StandardScaler()
sc.fit(X_train)
 
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [615]:
from sklearn.preprocessing import StandardScaler
 # 전체 데이터 정규화-pycaret용
sc = StandardScaler()
sc.fit(x)
 
x_std = sc.transform(x)

In [616]:
df_std = pd.DataFrame(x_std) # pycaret용

In [617]:
df_std.columns = ['dist_rank','genre_rank','direct_rank','네이버_기대지수','star_buzz','movie_buzz_naver','google_trend']

In [618]:
df_std['Target'] = y

In [619]:
df_std # 정규화된 pycaret용 데이터셋

,dist_rank,genre_rank,direct_rank,네이버_기대지수,star_buzz,movie_buzz_naver,google_trend,Target
0,-0.449913,-1.141880,0.337342,-0.312967,1.188551,-0.444762,0.393764,0
1,1.069617,0.874491,0.337342,-0.019480,-0.315426,-0.487298,-0.372524,0
2,-1.209678,0.874491,-0.908217,-0.402250,0.204078,-0.331334,0.082087,0
3,1.069617,0.874491,1.630808,-0.440252,-0.989919,-0.671618,-1.140796,0
4,-0.829795,0.874491,0.337342,-0.367224,-0.668458,-0.685797,2.182429,0
...,...,...,...,...,...,...,...,...
570,-0.829795,0.874491,-2.632837,1.108570,1.702314,2.348408,0.739188,1
571,-1.209678,0.874491,0.576873,0.769525,0.567156,1.001448,0.618091,0
572,-0.449913,-2.150065,-2.632837,0.106775,0.878571,4.616973,-1.140796,1
573,0.689734,-0.385741,1.918245,-0.430523,-0.938256,-0.402226,-0.229589,0


## 총 11개의 알고리즘 사용

### logistic regression

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

In [334]:
lr = LogisticRegression(penalty = 'l2', random_state=100)
lr.fit(X_train, y_train)
lr_pred1 = lr.predict(X_test)
print('acc',accuracy_score(lr_pred1, y_test)) 
print('f1',f1_score(y_test, lr_pred1))

acc 0.791907514450867
f1 0.4375


In [353]:
from sklearn.model_selection import KFold
kf = KFold(n_splits =  10, shuffle = True, random_state = 100) # fold 10번하기

lr = LogisticRegression(penalty = 'l2', random_state = 100)
acc_list = []
lr_pred = np.zeros((y_test.shape[0]))
for tr_idx, val_idx in kf.split(X_train, y_train) :
    tr_x, tr_y = X_train.iloc[tr_idx], y_train.iloc[tr_idx]
    val_x, val_y = X_train.iloc[val_idx], y_train.iloc[val_idx]
    
    lr.fit(tr_x, tr_y)
    
    pred = lr.predict(val_x)
    sub_pred = lr.predict(X_test)
    acc = accuracy_score(val_y, pred)
    
    acc_list.append(acc)
    
    lr_pred += (sub_pred / 10)

In [354]:
from sklearn.metrics import f1_score
print('acc 평균', np.mean(acc_list))
print('acc',accuracy_score(np.round(lr_pred), y_test)) 
print('f1 score', f1_score(np.round(lr_pred), y_test))

acc 평균 0.7885365853658536
acc 0.791907514450867
f1 score 0.4375


### linear discriminant

In [355]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [356]:
lda = LinearDiscriminantAnalysis(n_components=1)
lda.fit(X_train, y_train)
#lda.transform(X_train)
lda_pred1 = lda.predict(X_test)
print('acc', accuracy_score(lda_pred1, y_test))
print('f1',f1_score(y_test, lda_pred1))

acc 0.791907514450867
f1 0.4545454545454545


In [357]:
from sklearn.model_selection import KFold
kf = KFold(n_splits =  10, shuffle = True, random_state = 42) # fold 10번하기

lda = LinearDiscriminantAnalysis(n_components=1)
acc_list = []
lda_pred = np.zeros((y_test.shape[0]))
for tr_idx, val_idx in kf.split(X_train, y_train) :
    tr_x, tr_y = X_train.iloc[tr_idx], y_train.iloc[tr_idx]
    val_x, val_y = X_train.iloc[val_idx], y_train.iloc[val_idx]
    
    lda.fit(tr_x, tr_y)
    
    pred = lda.predict(val_x)
    sub_pred = lda.predict(X_test)
    acc = accuracy_score(val_y, pred)
    
    acc_list.append(acc)
    
    lda_pred += (sub_pred / 10)

In [358]:
print('acc 평균', np.mean(acc_list))
print('acc',accuracy_score(np.round(lda_pred), y_test)) 
print('f1 score', f1_score(np.round(lda_pred), y_test))

acc 평균 0.7907926829268292
acc 0.791907514450867
f1 score 0.4545454545454545


### rigde classifier

In [359]:
from sklearn.linear_model import RidgeClassifier

In [360]:
rc = RidgeClassifier(random_state = 100)
rc.fit(X_train, y_train)
rc_pred1 = rc.predict(X_test)
print("Accuracy:",accuracy_score(y_test, rc_pred1))
print("F1 score:",f1_score(y_pred, y_test))
# 좋은 결과가 나오지만 보팅에 넣을 수가 없음

Accuracy: 0.791907514450867
F1 score: 0.4375


In [361]:
from sklearn.model_selection import KFold
kf = KFold(n_splits =  10, shuffle = True, random_state = 100) # fold 10번하기

rc = RidgeClassifier(random_state=100)

acc_list = []
rc_pred = np.zeros((y_test.shape[0]))
for tr_idx, val_idx in kf.split(X_train, y_train) :
    tr_x, tr_y = X_train.iloc[tr_idx], y_train.iloc[tr_idx]
    val_x, val_y = X_train.iloc[val_idx], y_train.iloc[val_idx]
    
    rc.fit(tr_x, tr_y)
    
    pred = rc.predict(val_x)
    sub_pred = rc.predict(X_test)
    acc = accuracy_score(val_y, pred)
    
    acc_list.append(acc)
    
    rc_pred += (sub_pred / 10)

In [362]:
print('acc 평균', np.mean(acc_list))
print("Accuracy:",accuracy_score(np.round(rc_pred), y_test))
print('f1 score', f1_score(np.round(rc_pred), y_test))

acc 평균 0.7985365853658537
Accuracy: 0.7861271676300579
f1 score 0.4126984126984127


### KNN

In [251]:
from sklearn.neighbors import KNeighborsClassifier

In [252]:
knn = KNeighborsClassifier(n_neighbors = 11)

knn.fit(X_train_std, y_train)
y_pred = knn.predict(X_test_std)
print('acc',accuracy_score(y_pred, y_test))
print('f1',f1_score(y_test, y_pred))
# 0.7803 모든 피처용
# 0.768 act_n 뺏을 경우

acc 0.7630057803468208
f1 0.3278688524590164


In [253]:
from sklearn.model_selection import KFold
kf = KFold(n_splits =  10, shuffle = True, random_state = 100) # fold 10번하기

knn = KNeighborsClassifier(n_neighbors = 11)
acc_list = []
knn_pred = np.zeros((y_test.shape[0]))
for tr_idx, val_idx in kf.split(X_train_std, y_train) :
    tr_x, tr_y = X_train_std[tr_idx], y_train.iloc[tr_idx]
    val_x, val_y = X_train_std[val_idx], y_train.iloc[val_idx]
    
    knn.fit(tr_x, tr_y)
    
    pred = knn.predict(val_x)
    sub_pred = knn.predict(X_test_std)
    acc = accuracy_score(val_y, pred)
    
    acc_list.append(acc)
    
    knn_pred += (sub_pred / 10)

In [254]:
print('acc 평균', np.mean(acc_list))
print('acc',accuracy_score(np.round(knn_pred), y_test))
print('f1 score', f1_score(np.round(knn_pred), y_test))

acc 평균 0.7934146341463414
acc 0.7630057803468208
f1 score 0.3050847457627119


               정확도    F1 score
정규화하기 전
- 모든 피처 9개(screen수, act_n, genre, director, distributor, 네이버_movie_buzz, 구글_movie_buzz, star_buzz, 네이버_기대지수) 
  - 0.7803, 0.4242 -0.7687, 0.3750

  
- Screen 수 뺀, 피처 8개(act_n, genre, director, distributor, 네이버_movie_buzz, 구글_movie_buzz, star_buzz, 네이버_기대지수) 


정규화 한 후
- 모든 피처 9개(screen수, act_n, genre, director, distributor, 네이버_movie_buzz, 구글_movie_buzz, star_buzz, 네이버_기대지수)
   - 0.7630, 0.3492

### AdaBoost

In [830]:
from sklearn.ensemble import AdaBoostClassifier

In [831]:
abc = AdaBoostClassifier(n_estimators=50,
                         learning_rate=1, random_state=100)
abc = abc.fit(X_train, y_train)
y_pred = abc.predict(X_test)
print("Accuracy:",accuracy_score(y_test, y_pred))
print("F1 score:",f1_score(y_pred, y_test))

Accuracy: 0.7803468208092486
F1 score: 0.36666666666666664


In [181]:
from sklearn.model_selection import KFold
kf = KFold(n_splits =  10, shuffle = True, random_state = 100) # fold 10번하기

abc = AdaBoostClassifier(n_estimators=50,
                         learning_rate=1, random_state=100)
acc_list = []
abc_pred = np.zeros((y_test.shape[0]))
for tr_idx, val_idx in kf.split(X_train, y_train) :
    tr_x, tr_y = X_train.iloc[tr_idx], y_train.iloc[tr_idx]
    val_x, val_y = X_train.iloc[val_idx], y_train.iloc[val_idx]
    
    abc.fit(tr_x, tr_y)
    
    pred = abc.predict(val_x)
    sub_pred = abc.predict(X_test)
    acc = accuracy_score(val_y, pred)
    
    acc_list.append(acc)
    
    abc_pred += (sub_pred / 10)

In [182]:
print('acc 평균', np.mean(acc_list))
print('acc',accuracy_score(np.round(abc_pred), y_test))
print('f1 score', f1_score(np.round(abc_pred), y_test))

acc 평균 0.7585975609756097
acc 0.7687861271676301
f1 score 0.3548387096774193


               정확도    F1 score
정규화하기 전
- 모든 피처 9개(screen수, act_n, genre, director, distributor, 네이버_movie_buzz, 구글_movie_buzz, star_buzz, 네이버_기대지수) 
  - 0.7572, 0.3225

  
- Screen 수 뺀, 피처 8개(act_n, genre, director, distributor, 네이버_movie_buzz, 구글_movie_buzz, star_buzz, 네이버_기대지수) 


정규화 한 후
- 모든 피처 9개(screen수, act_n, genre, director, distributor, 네이버_movie_buzz, 구글_movie_buzz, star_buzz, 네이버_기대지수)
   - 0.7572, 0.3225

### random forest

In [37]:
from sklearn.ensemble import RandomForestClassifier

In [38]:
rfc = RandomForestClassifier(random_state=100)
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)
print('Model accuracy score with 10 decision-trees : {0:0.4f}'. format(accuracy_score(y_test, y_pred)))
print('f1',f1_score(y_test, y_pred))

Model accuracy score with 10 decision-trees : 0.7630
f1 0.38805970149253727


               정확도    F1 score
정규화하기 전
- 모든 피처 9개(screen수, act_n, genre, director, distributor, 네이버_movie_buzz, 구글_movie_buzz, star_buzz, 네이버_기대지수) 
  - 0.7630, 0.3880

  
- Screen 수 뺀, 피처 8개(act_n, genre, director, distributor, 네이버_movie_buzz, 구글_movie_buzz, star_buzz, 네이버_기대지수) 


정규화 한 후
- 모든 피처 9개(screen수, act_n, genre, director, distributor, 네이버_movie_buzz, 구글_movie_buzz, star_buzz, 네이버_기대지수)
   - 0.7630, 0.3880

### gradient boosting

In [32]:
from sklearn.ensemble import GradientBoostingClassifier

In [33]:
gbc = GradientBoostingClassifier(random_state=100, max_depth=3, learning_rate=0.01) 
gbc.fit(X_train, y_train)
y_pred = gbc.predict(X_test)
print('정확도{:.4f}'.format(accuracy_score(y_pred, y_test)))
print('f1',f1_score(y_test, y_pred))

정확도0.7572
f1 0.19230769230769232


In [34]:
from sklearn.model_selection import KFold
kf = KFold(n_splits =  10, shuffle = True, random_state = 42) # fold 10번하기

gbc = GradientBoostingClassifier(random_state=100, max_depth=3, learning_rate=0.01)
acc_list = []
gbc_pred = np.zeros((y_test.shape[0]))
for tr_idx, val_idx in kf.split(X_train, y_train) :
    tr_x, tr_y = X_train.iloc[tr_idx], y_train.iloc[tr_idx]
    val_x, val_y = X_train.iloc[val_idx], y_train.iloc[val_idx]
    
    gbc.fit(tr_x, tr_y)
    
    pred = gbc.predict(val_x)
    sub_pred = gbc.predict(X_test)
    acc = accuracy_score(val_y, pred)
    
    acc_list.append(acc)
    
    gbc_pred += (sub_pred / 10)

In [35]:
print('acc 평균', np.mean(acc_list))
print('정확도{:.4f}'.format(accuracy_score(np.round(gbc_pred), y_test)))
print('f1 score', f1_score(np.round(gbc_pred), y_test))

acc 평균 0.7834756097560975
정확도0.7572
f1 score 0.2222222222222222


               정확도    F1 score
정규화하기 전
- 모든 피처 9개(screen수, act_n, genre, director, distributor, 네이버_movie_buzz, 구글_movie_buzz, star_buzz, 네이버_기대지수) 
  - 0.7572, 0.1923, -0.7572, 0.2222

  
- Screen 수 뺀, 피처 8개(act_n, genre, director, distributor, 네이버_movie_buzz, 구글_movie_buzz, star_buzz, 네이버_기대지수) 


정규화 한 후
- 모든 피처 9개(screen수, act_n, genre, director, distributor, 네이버_movie_buzz, 구글_movie_buzz, star_buzz, 네이버_기대지수)
   - 0.7572, 0.1923

### Quadrantic Discriminant Analysis

In [730]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import f1_score

In [733]:
cqd = QuadraticDiscriminantAnalysis(store_covariance=True)
cqd.fit(X_train, y_train)
y_pred = cqd.predict(X_test)
print("Accuracy:",accuracy_score(y_test, y_pred))
print("F1 score:",f1_score(y_pred, y_test))

Accuracy: 0.7630057803468208
F1 score: 0.48101265822784806


In [734]:
from sklearn.model_selection import KFold
kf = KFold(n_splits =  10, shuffle = True, random_state = 42) # fold 10번하기

cqd = QuadraticDiscriminantAnalysis(store_covariance=True)
acc_list = []
cqd_pred = np.zeros((y_test.shape[0]))
for tr_idx, val_idx in kf.split(X_train, y_train) :
    tr_x, tr_y = X_train.iloc[tr_idx], y_train.iloc[tr_idx]
    val_x, val_y = X_train.iloc[val_idx], y_train.iloc[val_idx]
    
    cqd.fit(tr_x, tr_y)
    
    pred = cqd.predict(val_x)
    sub_pred = cqd.predict(X_test)
    acc = accuracy_score(val_y, pred)
    
    acc_list.append(acc)
    
    cqd_pred += (sub_pred / 10)

In [735]:
print('acc 평균', np.mean(acc_list))
print('정확도{:.4f}'.format(accuracy_score(np.round(cqd_pred), y_test)))
print('f1 score', f1_score(np.round(cqd_pred), y_test))

acc 평균 0.7685365853658536
정확도0.7688
f1 score 0.4871794871794872


               정확도    F1 score
정규화하기 전
- 모든 피처 9개(screen수, act_n, genre, director, distributor, 네이버_movie_buzz, 구글_movie_buzz, star_buzz, 네이버_기대지수) 
  - 0.76300, 0.4810 -0.7688,0.4871

  
- Screen 수 뺀, 피처 8개(act_n, genre, director, distributor, 네이버_movie_buzz, 구글_movie_buzz, star_buzz, 네이버_기대지수) 


정규화 한 후
- 모든 피처 9개(screen수, act_n, genre, director, distributor, 네이버_movie_buzz, 구글_movie_buzz, star_buzz, 네이버_기대지수)
   - 0.76300, 0.4810

### pycaret

In [363]:
from pycaret.classification import *
from sklearn.model_selection import train_test_split
train, test = train_test_split(df_fin, test_size=0.3, random_state=100)
cat_features = ['genre_rank', 'direct_rank','dist_rank']  # 범주 변수들 설정해줌, ,

In [364]:
setup_clf = setup(data = train, target= 'Target', fold_shuffle=True, categorical_features=cat_features)

,Description,Value
0,session_id,3376
1,Target,Target
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(402, 8)"
5,Missing Values,False
6,Numeric Features,4
7,Categorical Features,3
8,Ordinal Features,False
9,High Cardinality Features,False


In [365]:
top10 = compare_models(sort='F1', n_select=10, fold=10) 
# 모델들을 비교하여 가장 좋은 순으로 보여줌
# sort를 accuracy보다 F1으로 하는것이 더 좋은 f1score랑 accuracy를 낳음,fold 10번하는게 정확도가 올라감

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
nb,Naive Bayes,0.7541,0.6646,0.3452,0.4662,0.3817,0.2359,0.2480,0.0060
dt,Decision Tree Classifier,0.7154,0.5930,0.3690,0.3755,0.3599,0.1837,0.1889,0.0060
xgboost,Extreme Gradient Boosting,0.7368,0.6769,0.3024,0.4113,0.3368,0.1809,0.1895,0.0900
lda,Linear Discriminant Analysis,0.7260,0.6101,0.3024,0.4361,0.3334,0.1697,0.1877,0.0110
catboost,CatBoost Classifier,0.7899,0.7226,0.2214,0.6100,0.3048,0.2170,0.2626,1.0560
svm,SVM - Linear Kernel,0.6363,0.0000,0.4381,0.2626,0.2971,0.1031,0.1189,0.0170
ridge,Ridge Classifier,0.7543,0.0000,0.2405,0.5427,0.2954,0.1742,0.2135,0.0100
gbc,Gradient Boosting Classifier,0.7544,0.7144,0.2238,0.4483,0.2890,0.1605,0.1799,0.0470
lr,Logistic Regression,0.7616,0.6714,0.2095,0.3942,0.2576,0.1519,0.1681,0.0240
et,Extra Trees Classifier,0.7403,0.6422,0.1952,0.4283,0.2558,0.1225,0.1434,0.1200


In [366]:
tuned_top5 = [tune_model(i, fold=5) for i in top10[:5]]
# 상위 4개, 5개로 했을 때 제일 좋은 결과가 나옴

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7719,0.7972,0.0000,0.0000,0.0000,0.0000,0.0000
1,0.7679,0.6744,0.0000,0.0000,0.0000,0.0000,0.0000
2,0.7679,0.8032,0.0000,0.0000,0.0000,0.0000,0.0000
3,0.7857,0.7652,0.0000,0.0000,0.0000,0.0000,0.0000
4,0.7857,0.5900,0.0000,0.0000,0.0000,0.0000,0.0000
Mean,0.7758,0.7260,0.0000,0.0000,0.0000,0.0000,0.0000
SD,0.0082,0.0821,0.0000,0.0000,0.0000,0.0000,0.0000


In [367]:
blender_top5 = blend_models(estimator_list=tuned_top5) #보팅(voting)해줌

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7241,0.7532,0.0000,0.0000,0.0000,-0.0642,-0.1066
1,0.7143,0.7483,0.0000,0.0000,0.0000,-0.0667,-0.1111
2,0.7143,0.6939,0.1429,0.3333,0.2000,0.0588,0.0667
3,0.7857,0.6439,0.0000,0.0000,0.0000,0.0000,0.0000
4,0.7857,0.6894,0.1667,0.5000,0.2500,0.1600,0.1931
5,0.8214,0.7803,0.1667,1.0000,0.2857,0.2391,0.3685
6,0.7500,0.8712,0.0000,0.0000,0.0000,-0.0652,-0.1005
7,0.8214,0.5076,0.1667,1.0000,0.2857,0.2391,0.3685
8,0.8571,0.6061,0.3333,1.0000,0.5000,0.4400,0.5311
9,0.8571,0.7727,0.3333,1.0000,0.5000,0.4400,0.5311


In [368]:
final_model = finalize_model(blender_top5)
prediction = predict_model(final_model, data=test)

In [369]:
prediction['Target'] = prediction['Target'].astype(int)
prediction['Label'] = prediction['Label'].astype(float)
prediction['Label'] = prediction['Label'].astype(int)

In [370]:
from pycaret.utils import check_metric
check_metric(prediction['Target'], prediction['Label'], metric = 'Accuracy') 

#10개 폴드, sort = F1 기준으로 했을 때,제일 좋은 결과가 나옴 
# = 정확도 0.7977, F1score 0.4775

# 0.763

0.7514

In [371]:
# F1score
check_metric(prediction['Target'], prediction['Label'], metric = 'F1') #0.3051

0.2712

In [372]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

from sklearn.model_selection import KFold
kf = KFold(n_splits =  10, shuffle = True, random_state = 100) # fold 10번하기

acc_list = []
py_pred = np.zeros((y_test.shape[0]))
for tr_idx, val_idx in kf.split(X_train, y_train) :
    tr_x, tr_y = X_train.iloc[tr_idx], y_train.iloc[tr_idx]
    val_x, val_y = X_train.iloc[val_idx], y_train.iloc[val_idx]
    
    final_model.fit(tr_x, tr_y)
    
    pred = final_model.predict(val_x)
    sub_pred = final_model.predict(X_test)
    acc = accuracy_score(val_y, pred)
    
    acc_list.append(acc)
    
    py_pred += (sub_pred / 10)

In [373]:
print('acc 평균:', np.mean(acc_list)) #0.771
print('acc:',accuracy_score(np.round(py_pred), y_test)) #0.7976, 0.8034
print('f1 score', f1_score(np.round(py_pred), y_test)) #0.5205, 0.5277

acc 평균: 0.7835365853658537
acc: 0.7456647398843931
f1 score 0.26666666666666666


                              정확도    F1 score
정규화하기 전
- 모든 피처 9개(screen수, act_n, genre, director, distributor, 네이버_movie_buzz, 구글_movie_buzz, star_buzz, 네이버_기대지수) 
  - 0.7746, 0.4935 -0.803, 0.5277
  
- Screen수 뺀, 피처 8개(act_n, genre, director, distributor, 네이버_movie_buzz, 구글_movie_buzz, star_buzz, 네이버_기대지수)       
  - 0.7803, 0.4412 -0.7687, 0.3548
  
- act_n 뺀, 피처 8개(screen수, genre, director, distributor, 네이버_movie_buzz, 구글_movie_buzz, star_buzz, 네이버_기대지수)       
  - 0.
- distributor(배급사) 뺀, 피처 8개(screen수, act_n, genre, director, 네이버_movie_buzz, 구글_movie_buzz, star_buzz, 네이버_기대지수) 


- genre 뺀, 피처 8개(screen수, act_n, distributor, director, 네이버_movie_buzz, 구글_movie_buzz, star_buzz, 네이버_기대지수) 

  
정규화 한 후
- 모든 피처 9개(screen수, act_n, genre, director, distributor, 네이버_movie_buzz, 구글_movie_buzz, star_buzz, 네이버_기대지수) 

  
- Screen수 뺀, 피처 8개(act_n, genre, director, distributor, 네이버_movie_buzz, 구글_movie_buzz, star_buzz, 네이버_기대지수) 


- act_n 뺀, 피처 8개(screen수, genre, director, distributor, 네이버_movie_buzz, 구글_movie_buzz, star_buzz, 네이버_기대지수) 
  - 
  
- distributor(배급사) 뺀, 피처 8개(screen수, act_n, genre, director, 네이버_movie_buzz, 구글_movie_buzz, star_buzz, 네이버_기대지수) 

  
- genre 뺀, 피처 8개(screen수, act_n, distributor, director, 네이버_movie_buzz, 구글_movie_buzz, star_buzz, 네이버_기대지수) 


                              정확도    F1 score
정규화하기 전
- 모든 피처 9개(screen수, act_n, genre, director, distributor, 네이버_movie_buzz, 구글_movie_buzz, star_buzz, 네이버_기대지수) 
  - fold-5, 상위 5개 모델(5개 이상, 이하가 같은 결과가 나오거나 낮음) -0.7919, 0.4857 -0.7745, 0.4507 - 제일 좋은 성적
  - fold-10, 상위 5개 모델 - 0.7803, 0.4412 -0.7572, 0.3437 
  
- Screen수 뺀, 피처 8개(act_n, genre, director, distributor, 네이버_movie_buzz, 구글_movie_buzz, star_buzz, 네이버_기대지수)       
  - fold-5, 상위 4개 모델 - 0.7572, 0.3 -0.7919, 0.4375    - 정확도 제일 좋은 성적
  - fold-5, 상위 5개 모델 - 0.7572, 0.2759 -0.7630, 0.3050  
  - fold-10,상위 4개 모델 - 0.7688, 0.2857 -0.7803 0.3448
  - fold-10,상위 5개 모델 - 0.7746, 0.4179 -0.7803, 0.4411 - F1 제일 좋은 성적
               
- act_n 뺀, 피처 8개(screen수, genre, director, distributor, 네이버_movie_buzz, 구글_movie_buzz, star_buzz, 네이버_기대지수)       
  - fold-5, 상위 4개 모델 - 0.7919, 0.4545 -0.7919, 0.4545
  - fold-5, 상위 5개 모델 - 0.7919, 0.4545 -0.7919, 0.4545 - 제일 좋은 성적
  - fold-10,상위 4개 모델 - 0.7688, 0.375  -0.7745, 0.3809
  - fold-10,상위 5개 모델 - 0.803,  0.4688 -0.7919, 0.4193 

- distributor(배급사) 뺀, 피처 8개(screen수, act_n, genre, director, 네이버_movie_buzz, 구글_movie_buzz, star_buzz, 네이버_기대지수) 
  - fold-5, 상위 4개 모델 - 0.7688, 0.333 -0.7687, 0.3103
  - fold-5, 상위 5개 모델 - 0.7688, 0.333 -0.7687, 0.3103
  - fold-10,상위 4개 모델 - 0.7861, 0.4127 -0.7919, 0.4375
  - fold-10,상위 5개 모델 - 0.7977, 0.4776, -0.7919, 0.4545 - 제일 좋은 성적

- genre 뺀, 피처 8개(screen수, act_n, distributor, director, 네이버_movie_buzz, 구글_movie_buzz, star_buzz, 네이버_기대지수) 
  - fold-5, 상위 4개 모델 
  - fold-5, 상위 5개 모델
  - fold-10,상위 4개 모델
  - fold-10,상위 5개 모델
  
정규화 한 후
- 모든 피처 9개(screen수, act_n, genre, director, distributor, 네이버_movie_buzz, 구글_movie_buzz, star_buzz, 네이버_기대지수) 
  - fold-4,상위 4개 모델 - 0.7977, 0.4776 -0.7976, 0.4761 - 제일 좋은 성적
  - fold-5,상위 5개 모델 - 0.7919, 0.4545, 0.7861, 0.4477
  - fold-10,상위 5개 모델 - 0.7803, 0.4412, 0.7861, 0.4637 
  
- Screen수 뺀, 피처 8개(act_n, genre, director, distributor, 네이버_movie_buzz, 구글_movie_buzz, star_buzz, 네이버_기대지수) 
  - fold-5, 상위 4개 모델 - 0.7746, 0.4935 -0.7630, 0.4383
  - fold-5, 상위 5개 모델 - 0.7746, 0.48   -0.7811, 0.5
  - fold-10,상위 4개 모델 - 0.7688, 0.444  -0.7687, 0.4285
  - fold-10,상위 5개 모델 - 0.7861, 0.4789 -0.7919, 0.5    - 제일 좋은 성적

- act_n 뺀, 피처 8개(screen수, genre, director, distributor, 네이버_movie_buzz, 구글_movie_buzz, star_buzz, 네이버_기대지수) 
  - fold-5, 상위 4개 모델 - 0.7803, 0.4571 -0.7803, 0.4571
  - fold-5, 상위 5개 모델 - 0.7919, 0.4706 -0.7803, 0.4411
  - fold-10,상위 4개 모델 - 0.7746, 0.4935 -0.7803, 0.4864
  - fold-10,상위 5개 모델 - 0.7746, 0.48   -0.7861, 0.4931 - 제일 좋은 성적
  
- distributor(배급사) 뺀, 피처 8개(screen수, act_n, genre, director, 네이버_movie_buzz, 구글_movie_buzz, star_buzz, 네이버_기대지수) 
  - fold-5, 상위 4개 모델 - 0.7803, 0.3871 -0.7803, 0.3871
  - fold-5, 상위 5개 모델 - 0.7919, 0.4545 -0.7976, 0.4615
  - fold-10,상위 4개 모델 - 0.7917, 0.4706 -0.7976, 0.4776 - 제일 좋은 성적
  - fold-10,상위 5개 모델 - 0.7977, 0.4776 -0.7919, 0.4545
  
- genre 뺀, 피처 8개(screen수, act_n, distributor, director, 네이버_movie_buzz, 구글_movie_buzz, star_buzz, 네이버_기대지수) 
  - fold-5, 상위 4개 모델 
  - fold-5, 상위 5개 모델
  - fold-10,상위 4개 모델
  - fold-10,상위 5개 모델
  

결론
- 정규화한 데이터가 대체적으로 조금 더 좋은 결과가 나옴
- fold 수는 그때 그때 5번 접는게 더 좋을 때가 있고 10번 접는게 더 좋은 결과가 나올 때가 있기 때문에 각각 해봐야 함
- compare_models(), sort는 accuracy순으로 하는 것 보다 f1 순위로 하는게 더 좋은 결과가 나옴
- tune_model() 모델 튜닝 할 때는 상위 4개 혹은 5개가 가장 좋은 결과를 가져옴
- 제일 잘 나온 모델은 

## K-fold한 것 보팅

In [223]:
a = (rc_pred+lr_pred+lda_pred)/3 # 상위 폴드한 예측값들을 더해서 그 개수만큼 나눠줌(보팅?)

In [374]:
a = (rc_pred+lr_pred+lda_pred+py_pred)/4

In [375]:
accuracy_score(np.round(a),y_test) # 그거에 대한 정확도 확인

0.7976878612716763

In [376]:
from sklearn.metrics import f1_score

f1_score(np.round(a), y_test)

0.46153846153846156

정규화 전
rc_pred+lr_pred+lda_pred
- 모든 피처 0.8034, 0.4848
- screen 제외 0.7745, 0.4179
- act_n 제외 0.7919, 0.4375
- genre 제외 0.7745, 0.4347
- direct 제외 0.7745, 0.48
- dist 제외 0.7745, 0.3999
- 네이버 기대지수 제외 0.7803, 0.4242

rc_pred+lr_pred+lda_pred+py_pred
- 모든 피처 0.8034, 0.4848
- screen 제외 0.7745, 0.4179
- act_n 제외 0.7976, 0.4615
- genre 제외 0.7687, 0.3933
- direct 제외 0.7630, 0.4383
- dist 제외 0.7745, 0.3999
- 네이버 기대지수 제외 0.7803, 0.4242
- star buzz 제외 0.7803, 0.4242
- 네이버 movie buzz 제외 0.7630, 0.2807
- 구글 트렌드 제외 0.7803, 0.4242

정규화 후
rc_pred+lr_pred+lda_pred
- 모든 피처 - 0.8034, 0.4848
- screen 제외 - 0.7803, 0.4242
- act_n 제외 - 0.7919, 0.4375 
- 배급사 제외 - 0.7803, 0.4062
- 장르 제외 - 0.7745, 0.4179
- 감독 제외 - 0.7630, 0.4383
- 네이버 기대지수 - 0.7803, 0.4242
- star_buzz - 0.7861, 0.4307
- 네이버 movie buzz - 0.7745, 0.3606
- google_trend - 0.7803, 0.4242
rc_pred+lr_pred+lda_pred+py_pred
- 모든 피처 - 0.8092, 0.5074
- screen 제외 - 0.7861, 0.4307
- act_n 제외 - 0.7919, 0.4375 

## 보팅

In [33]:
from sklearn.ensemble import VotingClassifier

In [229]:
vo_clf = VotingClassifier(estimators=[('LR',lr),('LD',lda),('Py',final_model)], voting='soft')

In [230]:
vo_clf.fit(X_train, y_train)
pred = vo_clf.predict(X_test)
print("Voting 분류기 정확도", accuracy_score(y_test, pred))
print('f1',f1_score(y_test, pred))
# 0.7745모든 피처
# 0.7976 act_n 뺏을 때(soft), 0.7803(hard)
# 0.7745 dist_rank 뺏을 경우
# 0.7745 genre_rank 뺏을 경우
# 0.7572 기대지수 뺏을 경우
# 0.7687 star_buzz 뺏을 경우
# 0.7398 move_buzz_naver 뺏을 경우
# 0.7683 google_tren 뺏을 경우

# 0.803, f1 0.4848, lr, ld, py

Voting 분류기 정확도 0.7687861271676301
f1 0.411764705882353


In [231]:
acc_list = []
f1_list = []
vo_pred = np.zeros((y_test.shape[0]))
for tr_idx, val_idx in kf.split(X_train, y_train) :
    tr_x, tr_y = X_train.iloc[tr_idx], y_train.iloc[tr_idx]
    val_x, val_y = X_train.iloc[val_idx], y_train.iloc[val_idx]
    
    vo_clf.fit(tr_x, tr_y)
    
    pred = vo_clf.predict(val_x)
    sub_pred = vo_clf.predict(X_test)
    acc = accuracy_score(val_y, pred)
    f1_list = f1_score(np.round(vo_pred), y_test)
    acc_list.append(acc)
    
    vo_pred += (sub_pred / 10)

In [232]:
np.mean(acc_list)

0.7985365853658537

In [233]:
accuracy_score(np.round(vo_pred), y_test)

0.7803468208092486

In [234]:
from sklearn.metrics import f1_score

f1_score(np.round(vo_pred), y_test)

0.42424242424242425

                              정확도    F1 score
정규화하기 전
- 모든 피처 9개(screen수, act_n, genre, director, distributor, 네이버_movie_buzz, 구글_movie_buzz, star_buzz, 네이버_기대지수) 
  - 0.8034, 0.4848, 0.8034, 0.4848
  
- Screen수 뺀, 피처 8개(act_n, genre, director, distributor, 네이버_movie_buzz, 구글_movie_buzz, star_buzz, 네이버_기대지수)   - 0.763, 0.4057, 0.7803, 0.4242    

- act_n 뺀, 피처 8개(screen수, genre, director, distributor, 네이버_movie_buzz, 구글_movie_buzz, star_buzz, 네이버_기대지수)   -  0.7976, 0.4615, 0.7919, 0.4545    


- distributor(배급사) 뺀, 피처 8개(screen수, act_n, genre, director, 네이버_movie_buzz, 구글_movie_buzz, star_buzz, 네이버_기대지수) 
 
 
- genre 뺀, 피처 8개(screen수, act_n, distributor, director, 네이버_movie_buzz, 구글_movie_buzz, star_buzz, 네이버_기대지수) 
  -  0.7861, 0.4477, 0.7861, 0.4477
- director 뺀, 피처 8개(screen수, act_n, distributor, director, 네이버_movie_buzz, 구글_movie_buzz, star_buzz, 네이버_기대지수)
  - 0.7745, 0.4507, 0.7745, 0.4507

정규화 한 후
- 모든 피처 9개(screen수, act_n, genre, director, distributor, 네이버_movie_buzz, 구글_movie_buzz, star_buzz, 네이버_기대지수) 

  
- Screen수 뺀, 피처 8개(act_n, genre, director, distributor, 네이버_movie_buzz, 구글_movie_buzz, star_buzz, 네이버_기대지수) 


- act_n 뺀, 피처 8개(screen수, genre, director, distributor, 네이버_movie_buzz, 구글_movie_buzz, star_buzz, 네이버_기대지수) 
  - 0.7919, 0.4375
  
- distributor(배급사) 뺀, 피처 8개(screen수, act_n, genre, director, 네이버_movie_buzz, 구글_movie_buzz, star_buzz, 네이버_기대지수) 


- genre 뺀, 피처 8개(screen수, act_n, distributor, director, 네이버_movie_buzz, 구글_movie_buzz, star_buzz, 네이버_기대지수) 
